In [20]:
import torch
from torch.autograd import Function
from torch.autograd import Variable as V
import torch as t
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython import display
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
device = torch.device("cuda")
device1 = torch.device("cpu")
import torch.optim as optim
import time
import pandas as pd
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn.modules import Module
from torch.nn.modules.lazy import LazyModuleMixin
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [27]:
#Mean Square Error
def MSE(pred,true):
    return np.mean((pred-true)**2)

#Root Mean Square Error
def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))

#Mean Absolute Error
def MAE(pred, true):
    return np.mean(np.abs(pred-true))

#Mean Absolute Percentage Error
def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

#Relative Squared Error
def RSE(pred, true):
    return np.sqrt(np.sum((true-pred)**2)) / np.sqrt(np.sum((true-true.mean())**2))

def CORR(pred, true):
    u = ((true-true.mean(0))*(pred-pred.mean(0))).sum(0) 
    d = np.sqrt(((true-true.mean(0))**2*(pred-pred.mean(0))**2).sum(0))
    return (u/d).mean(-1)

'''
def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))
'''

#Coefficient of Determination
def R2(pred, true):
    return r2_score(pred,true) 


def gamma(a):
    
    if a <= 0 and a %1 == 0:
        return float('inf')
    
    return math.gamma(a)



def matrix_fractional_differentiation_linear_plus(x,w,b,a):
    
    w_matrix_fractional_differentiation = x
    
    w_rows, w_cols = w.size()
    x_rows, x_cols = x.size()
    x = ((t.sum(x,dim=0)/x_rows).view(1,-1))
   
    y = F.linear(x,w.T,b)  
    f1 = t.mul(x,w.T).T
    c_partial = y - f1
    
    #x
    xx = t.abs(x)
    x_partial = t.mul(c_partial.T,xx**(-a)/gamma(1-a)).T
    x_main = t.mul(w.T,xx**(1-a)/gamma(2-a)).T
    
    '''
    #w
    ww = t.abs(w)
    w_partial = t.mul(c_partial,ww**(-a)/gamma(1-a))
    w_main = t.mul(x,ww.T**(1-a)/gamma(2-a)).T 
    w_matrix_fractional_differentiation = (t.sum(w_main+w_partial,dim=1)/w_cols).view(1,-1)
    '''
    #b
    bb = t.abs(b)
    b_c_partial = y - b
    b_partial = t.mul(b_c_partial,bb**(-a)/gamma(1-a))
    b_main = bb**(1-a)/gamma(2-a)
    
    return x_main+x_partial, w_matrix_fractional_differentiation, b_main+b_partial

#This is not an important Class; its purpose is solely to obtain the weight matrix W and bias b.
#We have rearranged the positions of 'in_features' and 'out_features' for ease of future use.
class Linear(Module):

    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device1, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((in_features, out_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight.T, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
    
    
def split(X,y):
    X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333,shuffle=False)
    return X_train,X_val,X_test,y_train,y_val,y_test


#ifogd_MSELoss
class MSELoss_fractional(Function):
    
    def forward(ctx,input1,target):
        loss = nn.MSELoss()
        ctx.save_for_backward(input1,target)
        return loss(input1,target)
    
    def backward(ctx,grad_outputs):
        input1,target = ctx.saved_tensors
        t0, t1 = t.tensor([0.]),t.tensor([1.])
        t0 = t0.to(device1)
        t1 = t1.to(device1)
        y1 = t.where(input1 - target <= 0, t0, t1)
        y_pred_f = t.abs(input1 - target)
        grad_output1 = gamma(3)*y_pred_f**(2-a)/gamma(3-a)*y1
        y2 = t.where(input1 - target > 0, t0, t1)
        grad_output2 = 2*(input1 - target)*y2

        grad_output = (grad_output1 + grad_output2)/input1.numel()
        
        return grad_output,None
    
#ifogd_Linear
class Linear_fractional(Function):
    
    def forward(ctx,input1,w,b):
        ctx.save_for_backward(input1,w,b)
        outputs = input1@w + b
        return outputs
    
    def backward(ctx,grad_outputs):
        input1,w,b = ctx.saved_tensors
        #w4:x_matrix_fractional_differentiation
        #x4:w_matrix_fractional_differentiation
        #b4:b_matrix_fractional_differentiation
        w4, x4, b4 = matrix_fractional_differentiation_linear_plus(input1,w,b,a)   #IFOGD
        
        x_grad = torch.mm(grad_outputs,w4.T)
        w_grad = torch.mm(x4.T,grad_outputs)
        b_grad = torch.mul(grad_outputs,b4)
        return x_grad,w_grad,b_grad
    

In [29]:
#GSPC，DJI，IXIC，FTSE，GDAXI，N225 data preprocessing
time1 = time.time()
slide_windows_size = 36  #i.e.,input length
stock_array = ['GSPC','DJI','IXIC','FTSE','GDAXI','N225']   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
#stock_array = ['ETTh1','ETTh2','ETTm1','ETTm2']   #ETTh1，ETTh2，ETTm1，ETTm2
for stock in stock_array:
    #stock = 'DJI'   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
    pre_days = 48     #i.e.,prediction lengths,48,96,192,384
    while pre_days <= 384:  
        df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
        del df_DJIA['Date']
        #del df_DJIA['date']        #ETT
        sca_DJIA = scaler.fit_transform(df_DJIA)
        label = t.tensor([row[4] for row in sca_DJIA][slide_windows_size:]).view(-1,1)
        #label = t.tensor([row[6] for row in sca_DJIA][slide_windows_size:]).view(-1,1)  #ETT
        sca_DJIA = t.tensor(sca_DJIA[:-pre_days])
        #label
        deq_DJIA = deque(maxlen=pre_days)
        DJIA_label = []
        for i in label:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==pre_days:
                DJIA_label.append(list(deq_DJIA))
        DJIA_label = t.Tensor(DJIA_label).view(-1,pre_days)
        #X
        deq_DJIA = deque(maxlen=slide_windows_size)
        DJIA_X = []
        for i in sca_DJIA:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==slide_windows_size:
                DJIA_X.append(list(deq_DJIA))
        DJIA_X = t.Tensor(DJIA_X)
        X_DJIA, y_DJIA = DJIA_X.to(device1), DJIA_label.to(device1)
        X_train,X_val,X_test,y_train,y_val,y_test = split(X_DJIA,y_DJIA)   #7:2:1

        class Net(nn.Module):
            def __init__(self,):
                super(Net,self).__init__()
                self.linear1 = Linear(slide_windows_size*5,128)
                #self.linear1 = Linear(slide_windows_size*7,128)  #ETT
                self.linear2 = Linear(128,64)
                self.linear3 = Linear(64,32)
                self.linear4 = Linear(32,pre_days)
                self.w1,self.b1 = self.linear1.weight,self.linear1.bias
                self.w2,self.b2 = self.linear2.weight,self.linear2.bias
                self.w3,self.b3 = self.linear3.weight,self.linear3.bias
                self.w4,self.b4 = self.linear4.weight,self.linear4.bias

            def forward(self,x):
                x = x.view(-1,slide_windows_size*5)
                #x = x.view(-1,slide_windows_size*7)   #ETT
                y2 = Linear_fractional.apply(x,self.w1,self.b1)   
                y3 = Linear_fractional.apply(y2,self.w2,self.b2)   
                y4 = Linear_fractional.apply(y3,self.w3,self.b3) 
                y5 = Linear_fractional.apply(y4,self.w4,self.b4)  
                return y5

        net = Net()
        torch.save(net.state_dict(), r'./model/Adagrad_Stock/'+stock+'_'+str(pre_days)+'_net.pth')    

        a = 0.6
        lr =5e-3   #adagrad:5e-3.
        epoch = 1000
        batch_size = 256
        train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11 = [],[],[],[],[]  #collecting train set loss
        while a < 1.1:
            a = round(a + 0.1,1)
            best_loss = 10
            net = Net()
            net.load_state_dict(torch.load(r'./model/Adagrad_Stock/'+stock+'_'+str(pre_days)+'_net.pth'))
            optimizer = torch.optim.Adagrad(net.parameters(),lr=lr)
            net.to(device1)
            for i in range(epoch):
                loss_sum = 0
                net.train()  
                start = np.random.randint(11,size=1)[0]  #Generate a random integer between 0 and 10
                while start+batch_size<len(X_train):
                    X_batch = X_train[start:start+batch_size]
                    y_batch = y_train[start:start+batch_size]
                    start = start+batch_size
                    out = net(X_batch)
                    loss = MSELoss_fractional.apply(out,y_batch) 
                    loss.backward()
                    loss_sum += loss
                    optimizer.step()
                    net.zero_grad()
                if a==0.7:
                    train_loss_07.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.8:
                    train_loss_08.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.9:
                    train_loss_09.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.0:
                    train_loss_10.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.1:
                    train_loss_11.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                net.eval()
                with torch.no_grad():
                    out = net(X_val)
                    MSE_val = MSE(y_val.cpu().detach().numpy(),out.cpu().detach().numpy())   #MSE，MAPE
                    if best_loss > MSE_val:
                        best_loss = MSE_val
                        torch.save(net.state_dict(), r'./model/Adagrad_Stock/'+stock+'_net_fractional_'+str(a).replace(".", "_")+'_'+str(pre_days)+'.pth')
                        print('epoch:{0}'.format(i+1)+'\n'+'Validation Loss:{0}'.format(MSE_val))
            print('the order'+str(a)+'finished!')
        if pre_days==48:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adagrad_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==96:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adagrad_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==192:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adagrad_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==384:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adagrad_Stock/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        pre_days = pre_days*2
time2 = time.time()
print(time2 - time1)

epoch:1
Validation Loss:0.4178689122200012
epoch:2
Validation Loss:0.37147340178489685
epoch:3
Validation Loss:0.31839385628700256
epoch:7
Validation Loss:0.2787119448184967
epoch:8
Validation Loss:0.1851358562707901
epoch:9
Validation Loss:0.13184261322021484
epoch:10
Validation Loss:0.12871654331684113
epoch:11
Validation Loss:0.10663269460201263
epoch:13
Validation Loss:0.10536249727010727
epoch:14
Validation Loss:0.09805043041706085
epoch:15
Validation Loss:0.09691781550645828
epoch:16
Validation Loss:0.09528102725744247
epoch:17
Validation Loss:0.09130335599184036
epoch:23
Validation Loss:0.09096573293209076
epoch:24
Validation Loss:0.08986400812864304
epoch:26
Validation Loss:0.0893983244895935
epoch:28
Validation Loss:0.08675159513950348
epoch:35
Validation Loss:0.086445152759552
0.67668104
epoch:62
Validation Loss:0.08424507826566696
epoch:63
Validation Loss:0.08151347935199738
epoch:64
Validation Loss:0.08054330199956894
epoch:65
Validation Loss:0.07862041890621185
epoch:66
Va

epoch:236
Validation Loss:0.04761098325252533
epoch:237
Validation Loss:0.047607533633708954
epoch:238
Validation Loss:0.04750707000494003
epoch:240
Validation Loss:0.04737195745110512
epoch:241
Validation Loss:0.04736120626330376
epoch:242
Validation Loss:0.04731803759932518
epoch:243
Validation Loss:0.04723246023058891
epoch:244
Validation Loss:0.047219183295965195
epoch:245
Validation Loss:0.04709191620349884
0.21311161
0.12800662
epoch:313
Validation Loss:0.04706937447190285
epoch:314
Validation Loss:0.04696103185415268
epoch:315
Validation Loss:0.04692239314317703
epoch:316
Validation Loss:0.046854376792907715
epoch:318
Validation Loss:0.04682469740509987
epoch:319
Validation Loss:0.046807266771793365
epoch:320
Validation Loss:0.04674357920885086
epoch:321
Validation Loss:0.04658624157309532
epoch:322
Validation Loss:0.046555642038583755
epoch:324
Validation Loss:0.04654062166810036
epoch:325
Validation Loss:0.04641898348927498
epoch:327
Validation Loss:0.04629763960838318
epoch:3

epoch:108
Validation Loss:0.032429859042167664
epoch:109
Validation Loss:0.032344575971364975
epoch:110
Validation Loss:0.03225913271307945
epoch:111
Validation Loss:0.032177623361349106
epoch:112
Validation Loss:0.03209751844406128
epoch:113
Validation Loss:0.032030172646045685
epoch:114
Validation Loss:0.031959474086761475
epoch:115
Validation Loss:0.03187568485736847
epoch:116
Validation Loss:0.03180769830942154
epoch:117
Validation Loss:0.03173031657934189
epoch:118
Validation Loss:0.031664714217185974
epoch:119
Validation Loss:0.03158961609005928
epoch:120
Validation Loss:0.031522881239652634
epoch:121
Validation Loss:0.031459808349609375
epoch:122
Validation Loss:0.03140242397785187
epoch:123
Validation Loss:0.03133479878306389
epoch:124
Validation Loss:0.03127579763531685
epoch:125
Validation Loss:0.03121310845017433
epoch:126
Validation Loss:0.031160706654191017
epoch:127
Validation Loss:0.031095437705516815
epoch:128
Validation Loss:0.031043414026498795
epoch:129
Validation Lo

epoch:287
Validation Loss:0.02749640680849552
epoch:288
Validation Loss:0.02748679555952549
epoch:289
Validation Loss:0.027476951479911804
epoch:290
Validation Loss:0.02746226079761982
epoch:291
Validation Loss:0.027459969744086266
epoch:292
Validation Loss:0.02745286002755165
epoch:293
Validation Loss:0.02744515985250473
epoch:294
Validation Loss:0.02743525058031082
epoch:295
Validation Loss:0.027424611151218414
epoch:296
Validation Loss:0.02742217667400837
epoch:297
Validation Loss:0.027408111840486526
epoch:298
Validation Loss:0.027402125298976898
epoch:299
Validation Loss:0.02739906869828701
0.07404324
epoch:300
Validation Loss:0.02739270031452179
epoch:301
Validation Loss:0.027386678382754326
epoch:302
Validation Loss:0.027372675016522408
epoch:303
Validation Loss:0.027369685471057892
epoch:304
Validation Loss:0.027363987639546394
epoch:305
Validation Loss:0.027355743572115898
epoch:306
Validation Loss:0.02734542079269886
epoch:307
Validation Loss:0.027339568361639977
epoch:308
Va

epoch:495
Validation Loss:0.02651708759367466
epoch:496
Validation Loss:0.02651384472846985
epoch:497
Validation Loss:0.02651015669107437
epoch:498
Validation Loss:0.026506958529353142
epoch:499
Validation Loss:0.026503048837184906
0.07042142
epoch:500
Validation Loss:0.026497995480895042
epoch:501
Validation Loss:0.02649267204105854
epoch:503
Validation Loss:0.026489799842238426
epoch:504
Validation Loss:0.026485756039619446
epoch:506
Validation Loss:0.026483682915568352
epoch:507
Validation Loss:0.02647862210869789
epoch:508
Validation Loss:0.026476942002773285
epoch:509
Validation Loss:0.026473917067050934
epoch:510
Validation Loss:0.026466267183423042
epoch:512
Validation Loss:0.02646523155272007
epoch:514
Validation Loss:0.026460926979780197
epoch:516
Validation Loss:0.02645692601799965
epoch:517
Validation Loss:0.02645299956202507
epoch:518
Validation Loss:0.026448126882314682
epoch:519
Validation Loss:0.026443421840667725
epoch:520
Validation Loss:0.026439117267727852
epoch:521


...

epoch:78
Validation Loss:0.2872256338596344
epoch:79
Validation Loss:0.28713259100914
epoch:80
Validation Loss:0.28712865710258484
epoch:82
Validation Loss:0.287106454372406
epoch:83
Validation Loss:0.2870643138885498
epoch:84
Validation Loss:0.2870435118675232
epoch:85
Validation Loss:0.28692853450775146
epoch:87
Validation Loss:0.28692132234573364
epoch:88
Validation Loss:0.2868616580963135
epoch:90
Validation Loss:0.2868170738220215
epoch:91
Validation Loss:0.2867509722709656
epoch:92
Validation Loss:0.2867390215396881
epoch:93
Validation Loss:0.28664615750312805
epoch:94
Validation Loss:0.2866235375404358
epoch:95
Validation Loss:0.28658902645111084
epoch:96
Validation Loss:0.2865738570690155
epoch:97
Validation Loss:0.28648263216018677
epoch:98
Validation Loss:0.2864506244659424
epoch:99
Validation Loss:0.28644225001335144
0.9871359
epoch:100
Validation Loss:0.28637731075286865
epoch:101
Validation Loss:0.28627362847328186
epoch:102
Validation Loss:0.28626111149787903
epoch:103
Va

epoch:265
Validation Loss:0.27639636397361755
epoch:266
Validation Loss:0.2763911187648773
epoch:267
Validation Loss:0.2763496935367584
epoch:268
Validation Loss:0.2762715518474579
epoch:269
Validation Loss:0.27626144886016846
epoch:270
Validation Loss:0.2762223482131958
epoch:271
Validation Loss:0.27612781524658203
epoch:272
Validation Loss:0.2760617136955261
epoch:273
Validation Loss:0.2760266661643982
epoch:274
Validation Loss:0.2760201692581177
epoch:275
Validation Loss:0.27601727843284607
epoch:276
Validation Loss:0.27598774433135986
epoch:277
Validation Loss:0.27594879269599915
epoch:278
Validation Loss:0.2759043276309967
epoch:279
Validation Loss:0.2758328318595886
epoch:280
Validation Loss:0.2758200764656067
epoch:281
Validation Loss:0.2757943868637085
epoch:282
Validation Loss:0.27577638626098633
epoch:283
Validation Loss:0.27575862407684326
epoch:284
Validation Loss:0.27572616934776306
epoch:285
Validation Loss:0.27564290165901184
epoch:286
Validation Loss:0.2755652666091919


epoch:512
Validation Loss:0.26997432112693787
epoch:513
Validation Loss:0.2699006497859955
epoch:514
Validation Loss:0.26988163590431213
epoch:515
Validation Loss:0.2698802649974823
epoch:516
Validation Loss:0.2698380947113037
epoch:517
Validation Loss:0.2698328197002411
epoch:520
Validation Loss:0.269823282957077
epoch:521
Validation Loss:0.2697891294956207
epoch:522
Validation Loss:0.26974624395370483
epoch:527
Validation Loss:0.26973944902420044
epoch:529
Validation Loss:0.26967543363571167
epoch:530
Validation Loss:0.26964548230171204
epoch:531
Validation Loss:0.269606351852417
epoch:534
Validation Loss:0.2695605754852295
epoch:535
Validation Loss:0.2695308029651642
epoch:541
Validation Loss:0.2695148289203644
epoch:542
Validation Loss:0.269450843334198
epoch:543
Validation Loss:0.2694467306137085
epoch:545
Validation Loss:0.2694357633590698
epoch:547
Validation Loss:0.269377738237381
0.93490964
epoch:550
Validation Loss:0.26933836936950684
epoch:553
Validation Loss:0.2692507207393

epoch:884
Validation Loss:0.2644210755825043
epoch:891
Validation Loss:0.2644040882587433
epoch:892
Validation Loss:0.2643735408782959
epoch:896
Validation Loss:0.2643324136734009
epoch:898
Validation Loss:0.2642921209335327
0.925911
epoch:902
Validation Loss:0.26428160071372986
epoch:903
Validation Loss:0.26421478390693665
epoch:906
Validation Loss:0.26421207189559937
epoch:908
Validation Loss:0.26420289278030396
epoch:909
Validation Loss:0.2642021179199219
epoch:912
Validation Loss:0.2641867995262146
epoch:913
Validation Loss:0.26418378949165344
epoch:914
Validation Loss:0.2641560435295105
epoch:915
Validation Loss:0.2641507685184479
epoch:917
Validation Loss:0.26413822174072266
epoch:918
Validation Loss:0.26413777470588684
epoch:919
Validation Loss:0.2640741169452667
epoch:920
Validation Loss:0.26402968168258667
epoch:922
Validation Loss:0.2640187740325928
epoch:929
Validation Loss:0.26401200890541077
epoch:931
Validation Loss:0.2639661133289337
epoch:932
Validation Loss:0.263918519

epoch:27
Validation Loss:0.3055591285228729
1.937235
1.9130328
1.9269794
1.9306507
1.9269257
1.9220554
1.925002
1.9248278
1.9246536
1.9299502
1.9321322
1.9348185
1.9418758
1.946952
1.9498017
1.9513986
1.9565685
1.9608467
1.9603395
1.9686322
the order1.1finished!
N225 about 384 finished!
7592.580627918243
